In [7]:
import requests
import zipfile
import io
import os

!pip install little_mallet_wrapper



def download_mallet(url="https://github.com/mimno/Mallet/archive/refs/tags/v202108.zip", output_dir="Mallet_v202108"):
    """
    Downloads and extracts the MALLET toolkit from the specified URL.
    
    Parameters:
    - url (str): The URL to download the MALLET toolkit from.
    - output_dir (str): The directory where the toolkit will be extracted.
    """
    try:
        # Make a GET request to the URL
        response = requests.get(url)
        
        # Check if the request was successful
        if response.status_code == 200:
            # Extract the contents of the zip file into the output directory
            with zipfile.ZipFile(io.BytesIO(response.content)) as zip_ref:
                zip_ref.extractall(output_dir)
            print(f"Downloaded and extracted to {output_dir}")
        else:
            print(f"Failed to download file, status code: {response.status_code}")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
download_mallet()


You should consider upgrading via the '/Users/jashkina/Documents/LocResearch/plurals_system_paper/plurals_system_paper/venv/bin/python3 -m pip install --upgrade pip' command.
Downloaded and extracted to Mallet_v202108


In [19]:
pathy = "Mallet_v202108/Mallet-202108/bin/mallet"
import little_mallet_wrapper as lmw

descs = ballots['description'].tolist()


In [159]:
import pandas as pd
import textstat
import re


def make_prompt(row):
    overview = row['overview']
    description = row['description']
    yes_means = row['yes_means']
    no_means = row['no_means']

    s = f"DESCRIPTION\n{description}\n\nVOTING\n-{yes_means}\n\n-{no_means}\n\nDETAILED OVERVIEW\n{overview}"
    return s


def flesch_kincaid(text):
    if text:
        return textstat.flesch_kincaid_grade(text)
    else:
        return None
    

def clean_text(text):
    t = fix_number_spacing(text)
    t = remove_brackets(t)
    return t 

def fix_number_spacing(text):
    pattern = r'(\d)\.\s+(\d)'
    corrected_text = re.sub(pattern, r'\1.\2', text)
    return corrected_text

def remove_brackets(text):
    pattern = r'\[\d+\]'
    cleaned_text = re.sub(pattern, '', text)
    return cleaned_text

def sample_text(flesch_q, wc_q):
    return ballots.query(f'description_wc_q == {wc_q} & description_fk_q == {flesch_q}').sample(1)['description'].values[0]

ballots = pd.read_csv('../data/raw/2024-08-15_ballot_issues.csv')
print("Initial ballots", len(ballots))
ballots = ballots.query('is_valid == 1')
print("Ballots with valid metadata", len(ballots))

detailed_ballots = pd.read_csv("../data/raw/2024-08-15_detailed_ballot_issues.csv")
detailed_ballots = detailed_ballots.dropna(subset=['yes_means', 'no_means', 'overview'])
merged = pd.merge(ballots, detailed_ballots, on='url', how='inner', suffixes=('', '_detailed'))
print("Ballots with detailed information", len(merged))



merged['overview'] = merged['overview'].apply(clean_text)

merged['overview_wc'] = merged['overview'].apply(lambda x: textstat.lexicon_count(x, removepunct=True))
merged['overview_wc_q'] = pd.qcut(merged['overview_wc'], 4, labels=False)+1

merged['overview_fk'] = merged['overview'].apply(flesch_kincaid)
merged['overview_fk_q'] = pd.qcut(merged['overview_fk'], 4, labels=False)+1

merged['prompt'] = merged.apply(make_prompt, axis=1)


# fifty = ballots.groupby(by=['description_fk_q']).sample(10, random_state=1, replace=False)


Initial ballots 137
Ballots with valid metadata 136
Ballots with detailed information 124


In [161]:
print(merged['prompt'].sample(10).tolist()[0])

DESCRIPTION
Establish a single-subject rule for initiatives; increase the signature requirement for constitutional initiatives; and require constitutional initiatives to be approved at two elections

VOTING
-A "yes" vote supports this constitutional amendment to: establish a single-subject rule for initiatives (both statutory and constitutional), as determined by the secretary of state;increase the signature requirement for constitutional amendment initiatives from 4% of the resident population to 5% of the resident population of the state; andrequire proposed constitutional initiatives that have qualified for the ballot to be placed on the next primary election ballot, and, if approved, be placed on the next general election ballot, where it must be approved again to become effective.

-A "no" vote opposes this constitutional amendment to adopt a single-subject rule for initiatives and make changes to the signature requirement and election process for citizen-initiated constitutional 

In [146]:
print(merged.query('overview_wc_q == 1').sample(1)['overview'].values[0].strip())

This measure would change deadlines for filing initiative and referendum petition signatures to remove one week in order to allow one extra week for the secretary of state to certify ballot order and content and election officials' deadline to transmit ballots. Specifically, the amendment would: change initiative petition filing deadline to three months and one week prior to the election (rather than three months); change referendum petition filing to 83 days after adjournment of the legislative session (rather than 90 days); change the deadline for the text and title of every measure to be published from at least 15 days prior to the final date of voter registration for the election to 45 days before the election; and change deadlines for justices and judges to file declarations of intent to run for another term from not more than 6 months or less than 3 months prior to the general election before the expiration of the judge's term to not more than 6 months and one week or less than 3

In [132]:
s = """The measure would amend the Sales and Use Tax of 1955 to provide a sales tax exemption for both child and adult diapers, including disposable diapers. The sales and use tax is on the total revenue generated from the sale and storage, use, or consumption of tangible personal property in Nevada. [1] Nineteen states, along with the District of Columbia, exempt diapers from taxation. These states are California, Colorado, Connecticut, Florida, Indiana, Iowa, Louisiana, Maryland, Massachusetts, Minnesota, New Jersey, New York, North Dakota, Ohio, Pennsylvania, Rhode Island, Texas, Vermont, and Virginia. Twenty-six states did not exempt diapers from taxation, and five states—Alaska, Delaware, Montana, New Hampshire, and Oregon—do not have a state sales tax. [2] The Sales and Use Tax Act is the law addressing the sales tax in Nevada, including what products are exempted from the sales tax. Exemptions to the sales tax include farm machinery and equipment, food, prosthetic devices, newspapers, and other items. Recent exemptions approved by voters included feminine products and durable medical equipment, oxygen delivery equipment, and mobility enhancing equipment, which were both approved by voters in 2018. [3]"""

In [133]:
print(s.strip())

The measure would amend the Sales and Use Tax of 1955 to provide a sales tax exemption for both child and adult diapers, including disposable diapers. The sales and use tax is on the total revenue generated from the sale and storage, use, or consumption of tangible personal property in Nevada. [1] Nineteen states, along with the District of Columbia, exempt diapers from taxation. These states are California, Colorado, Connecticut, Florida, Indiana, Iowa, Louisiana, Maryland, Massachusetts, Minnesota, New Jersey, New York, North Dakota, Ohio, Pennsylvania, Rhode Island, Texas, Vermont, and Virginia. Twenty-six states did not exempt diapers from taxation, and five states—Alaska, Delaware, Montana, New Hampshire, and Oregon—do not have a state sales tax. [2] The Sales and Use Tax Act is the law addressing the sales tax in Nevada, including what products are exempted from the sales tax. Exemptions to the sales tax include farm machinery and equipment, food, prosthetic devices, newspapers, 

In [200]:
s = """Given a ballot initative, deliberate with others to decide how to vote. Answer in 50 words.

###

DESCRIPTION
Establish a single-subject rule for initiatives; increase the signature requirement for constitutional initiatives; and require constitutional initiatives to be approved at two elections

VOTING
-A "yes" vote supports this constitutional amendment to: establish a single-subject rule for initiatives (both statutory and constitutional), as determined by the secretary of state;increase the signature requirement for constitutional amendment initiatives from 4% of the resident population to 5% of the resident population of the state; andrequire proposed constitutional initiatives that have qualified for the ballot to be placed on the next primary election ballot, and, if approved, be placed on the next general election ballot, where it must be approved again to become effective.

-A "no" vote opposes this constitutional amendment to adopt a single-subject rule for initiatives and make changes to the signature requirement and election process for citizen-initiated constitutional initiatives.

DETAILED OVERVIEW
This amendment would make changes to the state's initiative process and requirements , including: establishing a single-subject rule for initiatives (both statutory and constitutional), as determined by the North Dakota Secretary of State; increasing the signature requirement for constitutional amendment initiatives from 4% of the resident population to 5% of the resident population of the state; and requiring proposed constitutional initiatives that have qualified for the ballot to be placed on the next primary election ballot in June, and, if approved, be placed on the next general election ballot in November, where it must be approved again to become effective.  A single-subject rule is a state law that requires ballot initiatives to address a single subject, topic, or issue. There are 26 states that provide for at least one type of statewide citizen-initiated measure. Of those 26 states, 17 have single-subject rules. As of 2023, North Dakota was one of nine states that provide for at least one type of statewide citizen-initiated measure and that did not have a single-subject rule. The last ballot measure to provide for a single-subject rule for initiatives was Arizona Proposition 129 , which was approved by a vote of 55% of voters in favor and 45% opposed. Nevada is the only state where initiated constitutional amendments must be approved at two consecutive elections. This does not apply to legislatively referred constitutional amendments, which must be approved twice by the legislature (with a majority vote) but only once by the state’s voters. Since the double election requirement was created in 1962, there have been 15 citizen-initiated constitutional amendments that passed at the first election and appeared on the ballot again at the next election. Of the 15 measures, 13 were passed at their second elections (86.66%) and two failed (13.33%). Question 9 (1982) was rejected with 58.77% of the vote after it was approved with 70.31% of the vote in 1980 . Question 3 (2018) was rejected with 67.05% of the vote after it was approved with 72.36% of the vote in 2016 ."""

In [201]:
from plurals.agent import Agent
from plurals.deliberation import Debate , Chain

a1 = Agent(model='gpt-3.5-turbo')
a2 = Agent(model='gpt-4o-mini')
a3 = Agent(model='gpt-4o')
chain = Chain(agents=[a1, a2], task=s, combination_instructions='empathetic', cycles=2)

chain.process()

In [204]:
for x in chain.responses:
    print(x)
    print()

I would vote "yes" on this ballot initiative. Establishing a single-subject rule for initiatives ensures clarity and focus on each proposal. Increasing the signature requirement for constitutional initiatives and requiring approval at two elections can help ensure that only well-supported and considered measures become law, promoting more thoughtful decision-making.

I believe a "yes" vote is crucial for ensuring our voices are truly heard. Implementing a single-subject rule fosters clarity, letting us focus on important issues. The increased signature requirement and double-election approval sound daunting, but they safeguard our democracy, helping us avoid rushed decisions. For underrepresented groups, this could create more opportunities to prioritize their concerns through clear, well-supported initiatives. While the changes may seem restrictive, they promote thoughtful engagement, ensuring that every voice, especially those often overlooked, has the time and space to resonate with

In [207]:
from plurals.agent import Agent
from plurals.deliberation import Debate , Chain

a1 = Agent(model='gpt-3.5-turbo')
a2 = Agent(model='gpt-4o-mini')
a3 = Agent(model='gpt-4o')
chain = Chain(agents=[a1, a2], task=s, combination_instructions='rational', cycles=2)

chain.process()

In [208]:
for x in chain.responses:
    print(x)
    print()

I would vote "Yes" on this initiative. Establishing a single-subject rule for initiatives can help ensure clarity and prevent misleading proposals. Increasing the signature requirement for constitutional initiatives and requiring approval at two elections can enhance the legitimacy of the process and ensure that significant changes have strong public support before becoming law.

I support voting "Yes" on this initiative. Implementing a single-subject rule for ballot initiatives will promote clarity and reduce confusion. Increasing the signature requirement ensures only well-supported efforts reach the ballot, while the double approval process confirms ongoing public support for significant changes. These measures strengthen democratic participation and ensure initiatives represent broad consensus.

I support voting "Yes" on this initiative. Implementing a single-subject rule enhances transparency and prevents unrelated issues from being bundled together. Raising the signature requirem

In [203]:
print(chain.agents[0].history[1]['response'])

In the heart of our community, the decision on this initiative resonates deeply. Imagine a single mother, juggling work and home, longing for initiatives that truly address her struggles. For marginalized voices often overlooked, a "yes" vote signifies hope and inclusivity. Let's embrace these changes as a promise of a more just and responsive democracy.
